In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://security.ubuntu.com/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-20 19:40:09--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.60MB/s    in 0.6s    

2022-07-20 19:40:10 (1.60 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
drop_na_df = df.dropna()
drop_na_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

In [6]:
#vine_df
vine_df = drop_na_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
#dataframe with total votes greater than or equal to 20
filtered_vine_df=vine_df.filter("total_votes>=20")
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R2JK5Y8D5MXAGP|          1|            5|        121|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1H5Y9Z2DHRNVO|          1|            4|         95|   N|                Y|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|     

Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

In [7]:
helpful_reviews_df = filtered_vine_df.filter("(helpful_votes/total_votes)>=0.5")
helpful_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|                Y|
|R2IXCZCSUKH8FB|          3|           78|         82|   N|                Y|
|R25RB38U9LL8OE|          5|           24|         29|   N|     

Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

In [8]:
vine_helpful_reviews_df=helpful_reviews_df.filter("vine=='Y'")
vine_helpful_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3BOQTLUU3Y4L8|          4|           42|         55|   Y|                N|
| RWF03LXVXC22A|          3|          396|        445|   Y|                N|
| REPU28WG1VZUE|          5|          242|        281|   Y|                N|
|R1D6REC9HPJVQY|          4|           31|         41|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

In [9]:
not_vine_helpful_reviews_df=helpful_reviews_df.filter("vine=='N'")
not_vine_helpful_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|                Y|
|R2IXCZCSUKH8FB|          3|           78|         82|   N|                Y|
|R25RB38U9LL8OE|          5|           24|         29|   N|     

Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)

In [10]:
total_num_helpful_reviews = helpful_reviews_df.count()
total_num_helpful_reviews

1068

vine reviews


In [11]:
total_num_helpful_vine_reviews = vine_helpful_reviews_df.count()
total_num_helpful_vine_reviews

4

In [17]:
five_star_vine_reviews = vine_helpful_reviews_df.filter("star_rating=='5'").count()
five_star_vine_reviews

1

In [20]:
percent_five_star_vine_reviews = (five_star_vine_reviews/total_num_helpful_vine_reviews)*100
percent_five_star_vine_reviews

25.0

None-Vine Reviews

In [12]:
total_num_helpful_not_vine_reviews = not_vine_helpful_reviews_df.count()
total_num_helpful_not_vine_reviews

1064

In [18]:
five_star_not_vine_reviews = not_vine_helpful_reviews_df.filter("star_rating=='5'").count()
five_star_not_vine_reviews

527

In [19]:
percent_five_star_not_vine_reviews = (five_star_not_vine_reviews/total_num_helpful_not_vine_reviews)*100
percent_five_star_not_vine_reviews

49.53007518796993